<a href="https://colab.research.google.com/github/ulfboge/wildlands-league-ecosystem/blob/main/scripts/notebokks/deforestation_minimal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deforestation Analysis for Algonquin Park

This notebook analyzes historical deforestation patterns in Algonquin Park using:
1. Hansen Global Forest Change dataset (2000-2023)
2. GLAD Global Land Cover and Land Use Change (GLCLUC2020) dataset

The analysis applies Canada's definition of forest and generates outputs to assess carbon sequestration feasibility.

In [ ]:
# Install required packages
!pip install numpy pandas geopandas matplotlib scipy earthengine-api geemap folium

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import ee
import geemap
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Configure paths for Google Drive
BASE_DIR = '/content/drive/MyDrive/wildlands_league'
STUDY_AREA_PATH = os.path.join(BASE_DIR, 'Algonquin_Park_Layers.gpkg')
OUTPUT_DIR = BASE_DIR

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Initialize Earth Engine
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
# Load the study area
try:
    study_area = gpd.read_file(STUDY_AREA_PATH)
    print(f"Loaded study area from {STUDY_AREA_PATH}")

    # Convert to GeoJSON for Earth Engine
    study_area_json = study_area.to_crs("EPSG:4326").__geo_interface__

    # Create Earth Engine geometry
    study_area_ee = ee.FeatureCollection(study_area_json)

except Exception as e:
    logging.error(f"Error loading study area: {str(e)}")
    raise

In [ ]:
# Load Hansen dataset
hansen = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")

# Apply Canada's forest definition (>25% canopy cover)
treecover2000 = hansen.select(['treecover2000'])
forest_cover = treecover2000.gte(25)

# Get forest loss year (1-23, representing 2001-2023)
loss_year = hansen.select(['lossyear'])

In [ ]:
# Create a map
map_obj = geemap.Map()

# Add forest cover in 2000
map_obj.add_layer(
    forest_cover.clip(study_area_ee),
    {'palette': ['green']},
    'Forest Cover 2000'
)

# Add forest loss by year
map_obj.add_layer(
    loss_year.updateMask(loss_year.gt(0)).clip(study_area_ee),
    {
        'min': 1,
        'max': 23,
        'palette': ['red', 'orange', 'yellow', 'green']
    },
    'Forest Loss Year'
)

# Add study area boundary
map_obj.add_layer(study_area_ee, {}, 'Study Area')

# Center map on study area
map_obj.center_object(study_area_ee, zoom=10)
map_obj

In [ ]:
# Calculate area in hectares
pixel_area = ee.Image.pixelArea().divide(10000)  # Convert to hectares

# Calculate total forest area in 2000
forest_area_2000 = forest_cover.multiply(pixel_area).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=study_area_ee,
    scale=30,
    maxPixels=1e12
).get('treecover2000').getInfo()

print(f"Forest area in 2000: {forest_area_2000:.2f} hectares")

In [ ]:
# Calculate forest loss by year
loss_by_year = {}
for year in range(1, 24):  # 2001-2023
    # Create mask for this year's loss
    this_year_loss = loss_year.eq(year).And(forest_cover)

    # Calculate area lost this year
    area_lost = this_year_loss.multiply(pixel_area).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=study_area_ee,
        scale=30,
        maxPixels=1e12
    ).get('lossyear').getInfo()

    loss_by_year[year + 2000] = area_lost

# Create DataFrame for loss by year
loss_df = pd.DataFrame({
    'Year': list(loss_by_year.keys()),
    'Area_Lost_Ha': list(loss_by_year.values())
})

# Calculate cumulative loss and percentage
loss_df['Cumulative_Loss_Ha'] = loss_df['Area_Lost_Ha'].cumsum()
loss_df['Percent_of_2000_Forest'] = (loss_df['Area_Lost_Ha'] / forest_area_2000) * 100
loss_df['Cumulative_Percent'] = loss_df['Cumulative_Loss_Ha'] / forest_area_2000 * 100

# Save to CSV
output_path = os.path.join(OUTPUT_DIR, "deforestation_loss_by_year.csv")
loss_df.to_csv(output_path, index=False)

print(f"Saved deforestation results to {output_path}")

In [ ]:
# Plot deforestation trends
fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(loss_df['Year'], loss_df['Area_Lost_Ha'], color='red', alpha=0.7)
ax.set_xlabel('Year')
ax.set_ylabel('Forest Loss (hectares)')
ax.set_title('Annual Forest Loss in Algonquin Park (2001-2023)')
ax.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "deforestation_trends.png"), dpi=300)
plt.show()